In [3]:
# !wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip

In [4]:
# import os
# import shutil
# import re
# src = 'data/train'
# dest = 'train/cats'
# src_files = os.listdir(src)

# # print(src_files)

# pattern = 'cat\.\d{1,4}\.jpg'
# first_10000_cat = [x for x in src_files if re.search(pattern, x)]
# for file_name in first_10000_cat:
#     full_file_name = os.path.join(src, file_name)
#     if os.path.isfile(full_file_name):
#         shutil.move(full_file_name, dest)



In [5]:
# src = 'data/train'
# dest = 'validation/cats'
# src_files = os.listdir(src)
# pattern = 'cat\.\d{5}\.jpg'
# last_2500_cat = [x for x in src_files if re.search(pattern, x)]
# for file_name in last_2500_cat:
#     full_file_name = os.path.join(src, file_name)
#     if os.path.isfile(full_file_name):
#         shutil.move(full_file_name, dest)

In [6]:
# src = 'data/train'
# dest = 'validation/dogs'
# src_files = os.listdir(src)
# pattern = 'dog\.\d{5}\.jpg'
# last_2500_dogs = [x for x in src_files if re.search(pattern, x)]
# for file_name in last_2500_dogs:
#     full_file_name = os.path.join(src, file_name)
#     if os.path.isfile(full_file_name):
#         shutil.move(full_file_name, dest)

In [7]:
# src = 'data/train'
# dest = 'train/dogs'
# src_files = os.listdir(src)
# pattern = 'dog\.\d{1,4}\.jpg'
# first_10000_dogs = [x for x in src_files if re.search(pattern, x)]
# for file_name in first_10000_dogs:
#     full_file_name = os.path.join(src, file_name)
#     if os.path.isfile(full_file_name):
#         shutil.move(full_file_name, dest)

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
import statistics

In [9]:
input_shape = (1, 150, 150, 3)

In [10]:
inputs = tf.keras.Input(shape=(150,150,3))

In [11]:
conv2d = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape[1:])(inputs)

2021-11-20 14:34:44.341053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-20 14:34:44.341092: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-20 14:34:44.341123: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mark-laptop): /proc/driver/nvidia/version does not exist
2021-11-20 14:34:44.341434: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
max_pooling_2d = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2d)

In [13]:
vectors = keras.layers.Flatten()(max_pooling_2d)

In [14]:
dense = keras.layers.Dense(64, activation='relu')(vectors)

In [15]:
output = keras.layers.Dense(1, activation='sigmoid')(dense)

In [16]:
model = keras.Model(inputs, output)

In [17]:
learning_rate = 0.002
momentum = 0.8
optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

In [18]:
loss = keras.losses.BinaryCrossentropy()

In [19]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

In [21]:
train_gen = ImageDataGenerator(rescale=1./255)

In [22]:
train_ds = train_gen.flow_from_directory(
    './train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [23]:
validation_ds = train_gen.flow_from_directory(
    './validation',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 5000 images belonging to 2 classes.


In [24]:
model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 23s 226ms/step - loss: 0.7000 - accuracy: 0.5210 - val_loss: 0.6898 - val_accuracy: 0.5370
Epoch 2/10
100/100 [==============================] - 22s 222ms/step - loss: 0.6891 - accuracy: 0.5435 - val_loss: 0.6874 - val_accuracy: 0.5420
Epoch 3/10
100/100 [==============================] - 23s 234ms/step - loss: 0.6860 - accuracy: 0.5405 - val_loss: 0.6874 - val_accuracy: 0.5400
Epoch 4/10
100/100 [==============================] - 23s 230ms/step - loss: 0.6798 - accuracy: 0.5720 - val_loss: 0.6785 - val_accuracy: 0.5930
Epoch 5/10
100/100 [==============================] - 22s 223ms/step - loss: 0.6764 - accuracy: 0.5650 - val_loss: 0.6670 - val_accuracy: 0.5990
Epoch 6/10
100/100 [==============================] - 22s 221ms/step - loss: 0.6699 - accuracy: 0.5880 - val_loss: 0.6661 - val_accuracy: 0.5780
Epoch 7/10
100/100 [==============================] - 22s 223ms/step - loss: 0.6691 - accuracy: 0.5910 - val_loss: 0.6595 - val_ac

In [25]:
statistics.median(model.history.history['accuracy'])

0.5800000131130219

In [26]:
statistics.stdev(model.history.history['loss'])

0.017426732384670483

## Question 4

In [27]:
train_gen_aug = ImageDataGenerator(
rescale=1./255, 
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

In [28]:
train_ds = train_gen_aug.flow_from_directory(
    './train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [29]:
# validation_ds = train_gen.flow_from_directory(
#     './validation',
#     target_size=(150, 150),
#     batch_size=20,
#     class_mode='binary'
# )

In [30]:
model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 26s 261ms/step - loss: 0.6596 - accuracy: 0.5955 - val_loss: 0.6774 - val_accuracy: 0.5610
Epoch 2/10
100/100 [==============================] - 26s 256ms/step - loss: 0.6666 - accuracy: 0.5765 - val_loss: 0.6410 - val_accuracy: 0.6050
Epoch 3/10
100/100 [==============================] - 26s 259ms/step - loss: 0.6650 - accuracy: 0.5835 - val_loss: 0.6583 - val_accuracy: 0.6020
Epoch 4/10
100/100 [==============================] - 25s 252ms/step - loss: 0.6604 - accuracy: 0.5940 - val_loss: 0.6293 - val_accuracy: 0.6310
Epoch 5/10
100/100 [==============================] - 25s 253ms/step - loss: 0.6642 - accuracy: 0.5955 - val_loss: 0.6355 - val_accuracy: 0.6290
Epoch 6/10
100/100 [==============================] - 26s 258ms/step - loss: 0.6611 - accuracy: 0.6000 - val_loss: 0.6730 - val_accuracy: 0.5750
Epoch 7/10
100/100 [==============================] - 25s 253ms/step - loss: 0.6532 - accuracy: 0.6000 - val_loss: 0.6329 - val_ac

In [31]:
statistics.mean(model.history.history['val_loss'])

0.6472484052181244

In [32]:
statistics.mean(model.history.history['val_accuracy'][5:])

0.6087999939918518